In [3]:
# disable compiler warnings
import os

# imports 
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_text as tf_text
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers import Dense
from typing import List
import datetime
from tqdm.notebook import tqdm


from tensorflow.python.client import device_lib
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'  # FATAL
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

Num GPUs Available:  []


In [4]:
file_path = "bible.txt"

with open(file_path, "r") as f:
    text = f.read().lower()

In [20]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
                                                    num_words=None,
                                                    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                                    lower=True,
                                                    split=' ',
                                                    char_level=False,
                                                    oov_token=None,
                                                    analyzer=None,
                                                )
# split into tokens
tokenizer.fit_on_texts([text])

# remember what numbers represent what words
word_index = tokenizer.word_index

# Encode training data sentences into sequences
train_sequences = tokenizer.texts_to_sequences([text])[0]

# only take most common 10000 words
train_sequences = [word for word in train_sequences if word <= 10000]

sequence_length = 4
dataset = tf.data.Dataset.from_tensor_slices(train_sequences)
dataset = dataset.window(sequence_length,drop_remainder=True)
batch_windows = lambda x: x.batch(sequence_length).get_single_element()
# this is new
split_sequence = lambda x: (x[:-1], x[-1])
dataset = dataset.map(batch_windows).map(split_sequence)

dataset = dataset.shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE)
for seq in dataset.take(1):
    tf.print(seq)


vocabulary_size=10000
embedding_dim=64
embedding_lookup = tf.keras.layers.Embedding(input_dim=vocabulary_size, output_dim=embedding_dim)

([[116 1869 213]
 [29 629 2]
 [39 10 49]
 ...
 [247 2 188]
 [95 1 488]
 [4 1681 2]], [3 29 686 ... 2 2 597])


In [21]:
class SkipGram(tf.keras.layers.Layer):
    def __init__(self, optimizer=tf.keras.optimizers.Adam()):
        super(SkipGram, self).__init__()
        




[[36 26 27 26]
 [47 1 35 23]
 [23 40 42 30]
 ...
 [36 1 23 36]
 [27 26 1 45]
 [43 1 29 23]]
